# EEG DTF Analysis: Reference Electrode Effects

**SYNCCIN 2025 Summer School - EEG Reference Comparison**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jzygierewicz/SYNCCIN_2025_summer_school/blob/main/DTF_EEG_reference.ipynb)

---

**Learning Objectives**

This notebook demonstrates how different EEG reference schemes affect DTF connectivity analysis:

1. **Compare reference methods**: Linked ears vs. Common Average Reference (CAR)
2. **Understand reference artifacts**: How reference choice impacts connectivity patterns
3. **Apply to real EEG data recorded in 10-20 system**: Use actual alpha rhythm recordings

---

## Setup: Install Dependencies & Download Data

Install required packages and download EEG data and analysis modules.

In [ ]:
# Install required packages
!pip install joblib networkx  xmltodict neurokit2 -q

print("All packages installed successfully!")

## Download Required Files

Download EEG data and Python analysis modules from the GitHub repository.

In [ ]:
import urllib.request
import os

# Base URL for the GitHub repository
base_url = "https://raw.githubusercontent.com/jzygierewicz/SYNCCIN_2025_summer_school/main/"

# Files to download
files_to_download = [
    "EEG_alpha.joblib",  # Real EEG data with alpha rhythm
    "mtmvar.py",
    "utils.py"
]

print("Downloading files from GitHub repository...")

for file in files_to_download:
    if not os.path.exists(file):
        try:
            urllib.request.urlretrieve(base_url + file, file)
            print(f"Downloaded: {file}")
        except Exception as e:
            print(f"Error downloading {file}: {e}")
    else:
        print(f"Already exists: {file}")

print("\nAll files ready for analysis!")

## Import Libraries and Load Data

Import analysis libraries and load the EEG alpha rhythm dataset.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import joblib
import networkx as nx

# Local imports
from mtmvar import mvar_plot_dense, DTF_multivariate, multivariate_spectra
from utils import describe_dict

# Load EEG data
data = joblib.load('EEG_alpha.joblib')
print('EEG Data Contents:')
describe_dict(data)

N_chan = data['EEG'].shape[0]
Fs = data['Fs'].item()

# DTF Analysis with Linked Ears Reference

## Configure Analysis Parameters

Set up frequency range and model parameters for DTF analysis. We propose anlysing results for alpha and beta bands separately.

In [ ]:
# f = np.arange(15, 25, 0.1)  # beta
f = np.arange(7, 15, 0.1)   # alpha frequency range 7-15 Hz
N_f = f.shape[0]
max_p = 25  # Maximum model order
p_opt = None  # Let algorithm select optimal order
crit_type = 'AIC'  # Information criterion

## Multivariate DTF - Linked Ears Reference

Compute DTF using the original linked ears reference.

In [ ]:
S_multivariate = multivariate_spectra(data['EEG'], f, Fs, max_p, p_opt=p_opt, crit_type=crit_type)
DTF = DTF_multivariate(data['EEG'], f, Fs, max_p, p_opt=p_opt, crit_type=crit_type)

mvar_plot_dense(S_multivariate, DTF, f, 'From\n', 'To\n', data['ChanNames'],
                'DTF - LER', 'sqrt')
plt.show()

# DTF Analysis with Common Average Reference (CAR)

## Apply CAR and Handle Rank Deficiency

Rereference to Common Average Reference and handle the resulting rank deficiency.

In [ ]:
# Apply CAR transformation
data_car = data['EEG'] - np.mean(data['EEG'], axis=0, keepdims=True)

# Handle rank deficiency by replacing one channel with noise
# T3 is chosen as it showed minimal connectivity in linked ears analysis
t3_index = data['ChanNames'].index('T3')
data_car[t3_index, :] = np.random.randn(*data_car[t3_index, :].shape)

print(f"CAR applied. Channel {data['ChanNames'][t3_index]} replaced with noise to handle rank deficiency.")

## Multivariate DTF - CAR Reference

Compute DTF using the Common Average Reference.

In [ ]:
S_multivariate_CAR = multivariate_spectra(data_car, f, Fs, max_p, p_opt=p_opt, crit_type=crit_type)
DTF_CAR = DTF_multivariate(data_car, f, Fs, max_p, p_opt=p_opt, crit_type=crit_type)

mvar_plot_dense(S_multivariate_CAR, DTF_CAR, f, 'From\n', 'To\n', data['ChanNames'],
                'DTF - CAR', 'sqrt')
plt.show()

# Reference Comparison and Conclusions

## Key Observations

**Linked Ears Reference:**
- Traditional choice
- Generally provides cleaner connectivity patterns

**Common Average Reference (CAR):**
- Requires handling rank deficiency (one channel must be removed/replaced)
- Introduces a mixture of phases from all channels to all other channels
- May introduce artificial connectivity due to reference contamination
- Can create interpretation challenges


## Clinical Implications
- **Reference choice matters**: Different references can lead to different connectivity conclusions

## Methodological Notes
- Consider reference choice when comparing studies or interpreting results

---

**Educational Materials**: [SYNCCIN 2025 Repository](https://github.com/jzygierewicz/SYNCCIN_2025_summer_school)

**Contact**:
- **Author**: Jarosław Żygierewicz
- **Institution**: University of Warsaw  
- **Event**: SYNCCIN 2025 Summer School